In [1]:
%pip install gradio numpy matplotlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from IPython.lib.display import Tuple
from PIL import Image
import numpy as np
class Unit:
  def __init__(self, pos: Tuple, token: Image, label) -> None:
    self.pos = np.array(pos, dtype = int)
    self.token = token
    self.actions = []
    self.label = label

  def add_action(self, type):
    pass

  def getToken(self, size = None) -> Image: 
    if size is None:
      return self.token.copy()
    else:
      return self.token.copy().resize((size, size))
  
  def getLabel(self) -> str:
    return self.label


# testUnit = Unit((0, 0), Image.open('./Screenshot from 2023-10-08 23-05-12.png'))
# print(type(testUnit.getToken(100)))
# testUnit.getToken(100).show()

Generate a grid

In [3]:

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def gridGen(grid_size = 5, space = 10):
    square_size = space + 1
    image_size = grid_size * square_size + 1
    new_image = Image.new('RGB', (image_size, image_size), color = 'white')
    for i in range(0, image_size, square_size):
        hline = Image.new('RGB', (image_size, 1), color = 'black')
        new_image.paste(hline, (0, i))
    for i in range(0, image_size, square_size):
        vline = Image.new('RGB', (1, image_size), color = 'black')
        new_image.paste(vline, (i, 0))
    return new_image

# gridGen().show()

In [4]:
from typing import List
import gradio as gr
import itertools
import os

units: List[Unit] = []

def generate_field(gridScale, gridSize):
  field = gridGen(gridSize, gridScale)
  return field, gr.Tabs(visible=True)

# Define the function to add an image to the canvas
def add_token(image, x, y, label, gridScale, gridSize):
  units.append(Unit(pos = (x, y), token = image, label=label))
  return drawField(gridScale, gridSize)

# Define the function to move an image
def move_token(index, x, y, gridScale, gridSize):
  units[index].pos = np.array([x, y], dtype = int)
  return drawField(gridScale, gridSize)

def drawField(gridScale = None, gridSize = None):
  field = gridGen(gridSize, gridScale)
  for unit in units:
    token = unit.getToken(gridScale)
    position = (1 + unit.pos[0]*(gridScale + 1), 1 + unit.pos[1]*(gridScale + 1))
    field.paste(token, position, token)
  return field
    
def updateTokenList():
  new_choices = list(map(lambda x : x.getLabel(), units))
  new_choices = list(zip(new_choices, itertools.count()))
  print(new_choices)
  print(type(new_choices))
  return gr.Dropdown(new_choices, interactive=True, label='tokenID')

def Dropdown_list():
    new_options =  ['lion', 'ikon2']
    print(new_options)
    return gr.Dropdown(choices=new_options)

with gr.Blocks() as demo:
  with gr.Row():
    gridScale = gr.Slider(label="gridScale", value=30, minimum=10, maximum=150, step=1)
    gridSize = gr.Slider(label="gridSize", value=10, minimum=2, maximum=100, step=1)
  im_canvas = gr.Image()
  gridScale.change(drawField, inputs = [gridScale, gridSize], outputs=[im_canvas])


  with gr.Tabs(visible=True) as tabs:
    with gr.TabItem("Add Token") as tab1:
      with gr.Row():
        with gr.Column(min_width=10):
          x = gr.Number(label="x pos")
          y = gr.Number(label="y pos")
          label = gr.Text()
        im_in = gr.Image(image_mode='RGBA', type='pil')
      btn_add = gr.Button(value="Add token")
    btn_add.click(add_token, inputs=[im_in, x, y, label, gridScale, gridSize], outputs=[im_canvas])

    with gr.TabItem("Move Token") as tab2:
      with gr.Row():
          tokenID = gr.Dropdown(['t1', 't2'], label='tokenID')
          tab2.select(updateTokenList, outputs=tokenID)
          x_move = gr.Number(label="x pos")
          y_move = gr.Number(label="y pos")
      btn_move = gr.Button(value="Move token")
    btn_move.click(move_token, inputs=[tokenID, x_move, y_move, gridScale, gridSize], outputs=[im_canvas])


if __name__ == "__main__":
    demo.launch()

/home/genom10/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


[('Husk1', 0), ('Husk2', 1), ('Husk3', 2)]
<class 'list'>
[('Husk1', 0), ('Husk2', 1), ('Husk3', 2), ('Aboba', 3)]
<class 'list'>
